In [1]:
import os

os.environ['PYTHONPATH']

'.;C:\\Users\\Playdata\\ObjectDetection\\TF_oda2\\models;C:\\Users\\Playdata\\ObjectDetection\\TF_oda2\\models\\research'

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [3]:
PIPELINE_CONFIG_PATH = os.path.join('model', 'pipeline.config')
CHECK_POINT_PATH = os.path.join('model', 'checkpoint', 'ckpt-51')
LABEL_MAP_FILE_PATH = os.path.join('labelmap', 'label_map.pbtxt')

In [4]:
# pipline.config에 맞춰서 추출한 모델을 바탕으로 모델을 생성 

# pipeline.config를 조회
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
# pipeline.config의 model설정 정보를 넣어서 모델생성
detection_model = model_builder.build(model_config=config['model'], is_training=False)

# 모델에 학습시킨 checkpoint(weight)를 주입
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(CHECK_POINT_PATH).expect_partial()

In [5]:
# detection을 실행하는 함수

# 순전파 처리 함수에 @tf.function decorator를 선언하면 실행 속도가 빨라진다.
@tf.function
def detect_func(image):
    """
    매개변수로 object detection을 수행할 대상 image(Tensor)를 받아서 detection 처리.
    1. preprocessing(전처리): resize, normalization 작업
    2. detection(inference-추론)
    3. detection결과를 post processing : Non Maxinum Suppression
    4. post processing한 결과를 반환.
    """
    # 1 preprocessing
    image, shapes = detection_model.preprocess(image)
    # 2. 추론
    predict_dict = detection_model.predict(image, shapes)
    # 3. post processing
    result = detection_model.postprocess(predict_dict, shapes)
    return result

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_FILE_PATH)
print(type(category_index))
category_index

<class 'dict'>


{1: {'id': 1, 'name': 'one'},
 2: {'id': 2, 'name': 'two'},
 3: {'id': 3, 'name': 'three'},
 4: {'id': 4, 'name': 'four'},
 5: {'id': 5, 'name': 'five'}}

In [14]:
# 웹캠으로 부터 이미지를 받아서 추론한 결과를 화면에 보여주기.
cap = cv2.VideoCapture(0)
print(cap.isOpened())
# 웹캠의 WIDTH/HEIGHT 조회
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

True


In [13]:
while True:
    ret, frame = cap.read() #한 frame 읽기.
    
    if not ret:
        print("이미지를 읽지 못함")
        break
    
    frame = cv2.flip(frame, 1) #좌우 반전
    
    #BGR=>RGB (모델이 학습할 때 RGB 모드로 학습했기 때문에 같은 형식으로 변환)
    image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #0번 축을 늘린다. Tensor로 변환
    input_tensor = tf.convert_to_tensor(image_np[np.newaxis, ...], dtype=tf.float32)
    # 추론
    post_detection = detect_func(input_tensor) #전처리->추론->후처리
    
    num_detections = int(post_detection.pop('num_detections'))
    # 추론한 결과들을 num_detections 개수(detection한 물체의 개수) 만큼의 값만 남긴다. 결과가 Tensor로 반환되는 것을 ndarray로 변환.
    detections = {key:value[0, :num_detections].numpy() for key, value in post_detection.items()}

    # 새로 구성한 결과 dictionary(detections)에 num_detections 값을 추가.
    detections['num_detections'] = num_detections
    # detection_classes 는 검출한 box의 class 값을 label encoding 된 값으로 가진다. float32로 반환되는 것을 int로 변환 처리.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    MIN_CONF_THRESH = 0.7 # 물체가 있을 Confidence score가 0.5 이상인 bounding box만 나오도록 하겠다.
    image_np_with_detection = image_np.copy()  #detection한 원본 이미지의 카피본을 생성.
    img = viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detection, #추론한 원본이미지,
                    detections['detection_boxes'], #bounding box 좌표
                    detections['detection_classes'] + 1, #bouding box내의 물체 index (class확률에서 0은 첫번째 label, label map의 id는 1부터 시작 하기 때문에 +1)
                    detections['detection_scores'], #bounidng box내에 물체가 있을 확률(confidence score)
                    category_index, 
                    use_normalized_coordinates=True, #bounding box의 좌표들이 normalize되었는지 여부
                    max_boxes_to_draw=100, # 최대 몇개 박스를 칠 것인지 (기본: 20)
                    min_score_thresh=MIN_CONF_THRESH)  # Confidence socre가 얼마 이상인 bounding box만 나오도록 하겠다.
    
    # 결과 image를 RGB => BGR
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    print(img.shape)
    break
    #화면에 출력
    cv2.imshow('frame', img)
    if cv2.waitKey(1) > 0: #아무키나 입력하면
        break
        
cap.release()
cv2.destroyAllWindows()

## post processing 결과 확인

In [15]:
img = cv2.imread('one.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img[np.newaxis, ...]
input_tensor = tf.convert_to_tensor(img, dtype=tf.float32)

In [16]:
post_detection = detect_func(input_tensor)

In [17]:
post_detection.keys()

dict_keys(['detection_boxes', 'detection_scores', 'detection_classes', 'num_detections', 'raw_detection_boxes', 'raw_detection_scores', 'detection_multiclass_scores', 'detection_anchor_indices'])

In [19]:
post_detection['num_detections'].numpy() #Tensor => ndarray
# num_detections: 후처리로 최종 결과로 나온 bounding box의 개수. 
# 전체 bounding box에서 confidence score 순으로 내림 차순한 뒤 NMS 을 거쳐서 최종적으로 남은 bounding box의 개수

array([100.], dtype=float32)

In [20]:
post_detection['detection_boxes'].shape
# [1, 100, 4] => [추론한이미지개수, num_detections, 좌표(x,y,w,h)]

TensorShape([1, 100, 4])

In [21]:
# bbox에 물체가 있을 확률 = confidence score
post_detection['detection_scores'].shape
# [1, 100], [추론한이미지개수, num_detections]

TensorShape([1, 100])

In [22]:
post_detection['detection_scores'][0, :10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.99999934, 0.02000004, 0.01957524, 0.01314369, 0.01004648,
       0.00982314, 0.00981891, 0.00946924, 0.00946558, 0.00924185],
      dtype=float32)>

In [23]:
post_detection['detection_boxes'][0, 0]

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.6061671 , 0.40268105, 0.97964144, 0.5990967 ], dtype=float32)>

In [32]:
post_detection['detection_scores']

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[0.99999934, 0.02000004, 0.01957524, 0.01314369, 0.01004648,
        0.00982314, 0.00981891, 0.00946924, 0.00946558, 0.00924185,
        0.00861913, 0.00804833, 0.00766084, 0.00732458, 0.0073235 ,
        0.00729269, 0.00718403, 0.00665718, 0.00665098, 0.00640824,
        0.00625852, 0.00610211, 0.00593981, 0.00584751, 0.00546545,
        0.005299  , 0.00509086, 0.00506631, 0.00506613, 0.0050447 ,
        0.00500393, 0.00495923, 0.00493613, 0.00493127, 0.00485682,
        0.00482705, 0.00470507, 0.00462821, 0.00452715, 0.00442025,
        0.00435778, 0.00427103, 0.00426495, 0.00413018, 0.00411677,
        0.00411001, 0.00406948, 0.00404009, 0.00403646, 0.00398517,
        0.00397146, 0.00394303, 0.00394279, 0.00392681, 0.00380638,
        0.00378749, 0.00370955, 0.00363976, 0.00363174, 0.00363004,
        0.00358856, 0.00358668, 0.00356436, 0.00356168, 0.00354788,
        0.00352094, 0.00346696, 0.00344554, 0.00343415, 0.0034281 

In [29]:
post_detection['detection_classes'][0, 0] 

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [27]:
# 각 class 별 확률
post_detection['detection_multiclass_scores'].shape

TensorShape([1, 100, 6])

In [28]:
post_detection['detection_multiclass_scores'][0,0]

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([1.3113618e-03, 9.9999934e-01, 1.1473298e-03, 1.3917685e-03,
       1.7806906e-08, 5.2607059e-04], dtype=float32)>

In [31]:
post_detection['raw_detection_boxes'].shape

TensorShape([1, 12804, 4])